<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Itertools" data-toc-modified-id="Itertools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Itertools</a></span><ul class="toc-item"><li><span><a href="#Accumulate--(vs.--functools.reduce)" data-toc-modified-id="Accumulate--(vs.--functools.reduce)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Accumulate  (vs.  functools.reduce)</a></span></li><li><span><a href="#Compress" data-toc-modified-id="Compress-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Compress</a></span></li></ul></li></ul></div>

In [2]:
import itertools
import more_itertools
import sys

import numpy as np

In [3]:
print(sys.executable)
print(sys.version)

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]


## Itertools

### Accumulate  (vs.  functools.reduce)

Accumulates the results of some (binary) function, e.g. max or factorial. Returns an iterator with all intermediary results.

In [7]:
data = [3, 4, 1, 3, 5, 6, 9, 0, 1]

# Calculate the running max
result = itertools.accumulate(data, max)  

list(result)  # Returns an iterator

[3, 4, 4, 4, 5, 6, 9, 9, 9]

In [20]:
# Some more examples using operator (addition is default, if no function is passed)

import operator

print(list(itertools.accumulate(data, operator.mul)))
print(list(itertools.accumulate(data, operator.sub, initial=1000)))

[3, 12, 12, 36, 180, 1080, 9720, 0, 0]
[1000, 997, 993, 992, 989, 984, 978, 969, 969, 968]


Attention: If you don’t care about intermediate results, use `functools.reduce` , which keeps only the final value and is more memory efficient:

In [13]:
from functools import reduce

result = reduce(max, data)

result

9

<img src="diff_between_red_acc.jpg" width="50%" />

### Compress

One of many options to filtering sequences: 

`compress` takes an _iterable_ and a _boolean selector_ and returns an iterator containing items of the iterable where the corresponding element in the selector is True.

In [6]:
dates = [
    "2020-01-01",
    "2020-02-04",
    "2020-02-01",
    "2020-01-24",
    "2020-01-08",
    "2020-02-10",
    "2020-02-15",
    "2020-02-11",
]

counts = [1, 4, 3, 8, 0, 7, 9, 2]

bools = [n > 3 for n in counts]
result = itertools.compress(dates, bools)

list(result)  # Returns an iterator

['2020-02-04', '2020-01-24', '2020-02-10', '2020-02-15']